In [33]:

import numpy as np
import pandas as pd
import sys
import csv
from uneye2 import uneye

# Reading the dataset

In [34]:
SAMPLEING_FREQUENCY = 1000
data = pd.read_csv('train2.csv',encoding='utf-8', header=None)

#### Filter by subject

In [35]:
subjects = ["s2","s4","s8","s10","s16","s22","s23","s30"] 
data = data[data.iloc[:, 0].isin(subjects)]
print(data)

    0      1       2        3       4        5       6        7       8      \
2      s8  False -664.47  -806.21 -665.24  -807.56 -666.01  -808.91 -666.40   
6     s16  False -449.71  -151.24 -451.57  -148.07 -453.42  -146.49 -452.50   
10    s10  False -831.28  -242.49 -829.87  -242.49 -829.17  -243.73 -829.17   
12     s4  False  735.38   276.22  734.64   274.54  735.38   274.54  736.86   
19    s30  False -307.14  -827.31 -306.58  -828.29 -306.58  -829.27 -306.02   
..    ...    ...     ...      ...     ...      ...     ...      ...     ...   
805    s2   True   95.46   251.41   94.76   252.48   94.06   251.41   94.06   
816    s4   True -707.09  1202.15 -707.09  1203.83 -707.83  1205.51 -705.61   
829   s30  False  490.75  -202.69  491.39  -203.59  490.75  -205.41  490.11   
831    s2  False  917.65  1161.16  917.65  1161.16  918.46  1161.16  919.27   
834   s30  False  477.44  -238.09  476.80  -235.37  476.80  -232.65  476.15   

       9      ...  16374  16375  16376  16377  1637

In [36]:
sample1 = data.iloc[30,2:].dropna()
x = np.array(sample1[0::2], dtype=float)
y = np.array(sample1[1::2], dtype=float)

print(x)
print(y)

[876.08 876.08 876.08 ... 136.06 135.5  136.62]
[844.42 843.44 843.44 ... 392.82 389.88 387.92]


# UnEye Deep Learning model

In [37]:
model = uneye.DNN(weights_name="weights_dataset3", sampfreq= SAMPLEING_FREQUENCY)
uneye_predictions = model.predict(x, y)[0]
print(uneye_predictions)

[0. 0. 0. ... 0. 0. 0.]


In [38]:
import math

def distance_between_two_points(x1, y1, x2, y2):
  return  math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def get_fixations(x, y, predictions, sampfreq, clustering_distance = 50):
    last = predictions[0]
    counter = 0
    fixations_position = list()
    fixations_duration = list()
    for i in range(1, len(predictions)):
        if last != predictions[i]:
            if last == 0:
                if len(fixations_position) == 0 or distance_between_two_points(x[i], y[i], fixations_position[-1][0], fixations_position[-1][1]) > clustering_distance:
                    fixations_position.append([x[i], y[i]])
                    fixations_duration.append(counter/sampfreq)
                else:
                    fixations_duration[-1] = fixations_duration[-1] + counter/sampfreq
                    
            last = predictions[i]
            counter = 1
        else:
            counter = counter + 1
    return np.array(fixations_position), np.array(fixations_duration)

# Velocity Algorithm

In [39]:

def fixation_velocity(x, y, freq, max_velocity = 4200, clustering_distance = 50):
  fixations_postion_list = list()
  fixations_duration_list = list()
  fixation_start_index = 0
  fixation_started = False
  for i in range(1, len(x)):
    distance = distance_between_two_points(x[i], y[i], x[i-1], y[i-1])
    velocity = distance*freq
    if(distance>=0):
      if (velocity > max_velocity and fixation_started == True):
        # fixation is ended, and started a sacade.
        
        new_fixation_x = (x[fixation_start_index] + x[i])/2
        new_fixation_y = (y[fixation_start_index] + y[i])/2
        new_fixation_dur = (i - fixation_start_index) * 1/freq

        if(len(fixations_postion_list) == 0 or 
            distance_between_two_points(new_fixation_x, new_fixation_y,
                                        fixations_postion_list[-1][0], fixations_postion_list[-1][1]) > clustering_distance
          ):
          fixations_postion_list.append([new_fixation_x, new_fixation_y])
          fixations_duration_list.append(new_fixation_dur)
        else:
          fixations_postion_list[-1][0] = (fixations_postion_list[-1][0] + new_fixation_x)/2
          fixations_postion_list[-1][1] = (fixations_postion_list[-1][1] + new_fixation_y)/2
          fixations_duration_list[-1] = fixations_duration_list[-1] + new_fixation_dur

        fixation_started = False
      elif(velocity <= max_velocity and fixation_started == False):
        # fixation is started.
        fixation_start_index = i
        fixation_started = True
  return np.array(fixations_postion_list) , np.array(fixations_duration_list)

# Comparing UnEye and Velocity Algorithm

### Comparing by fixations positions

In [40]:
import plotly.graph_objects as go

def compare_algos(sample_number):
    sample1 = data.iloc[sample_number,2:].dropna()
    x = np.array(sample1[0::2], dtype=float)
    y = np.array(sample1[1::2], dtype=float)
    model = uneye.DNN(weights_name="weights_dataset3", sampfreq= SAMPLEING_FREQUENCY)
    uneye_predictions = model.predict(x, y)[0]

    uneye_fixations_position,  uneye_fixations_duration = get_fixations(x, y, uneye_predictions, SAMPLEING_FREQUENCY)
    vel_algo_fixations_postion , vel_algo_fixations_duration = fixation_velocity(x, y, SAMPLEING_FREQUENCY)

    uneye_bubbles_size = uneye_fixations_duration + np.absolute(np.min(uneye_fixations_duration))
    uneye_bubbles_size = (30 * (uneye_bubbles_size / np.max(uneye_bubbles_size))).astype(int)

    vel_algo_bubbles_size = vel_algo_fixations_duration + np.absolute(np.min(vel_algo_fixations_duration))
    vel_algo_bubbles_size = (30 * (vel_algo_bubbles_size / np.max(vel_algo_bubbles_size))).astype(int)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        name="Fixations from Velocity Algorithm",
        x=np.array(vel_algo_fixations_postion)[:,0].astype(int),
        y=np.array(vel_algo_fixations_postion)[:,1].astype(int),
        mode='markers',
        marker=dict(size=vel_algo_bubbles_size)
    ))
    fig.add_trace(go.Scatter(
        name="Fixations from Uneye",
        x=np.array(uneye_fixations_position)[:,0].astype(int),
        y=np.array(uneye_fixations_position)[:,1].astype(int),
        mode='markers',
        marker=dict(size=uneye_bubbles_size)
    ))

    fig.update_layout(title="UnEye vs Velocity Algorithm for Sample" + str(sample_number), title_x=0.45, legend=dict(x=0.3,y=-0.3))
    fig.show()

In [41]:
compare_algos(15)
compare_algos(23)
compare_algos(48)
compare_algos(53)

### Comparing by fixations measurement

In [42]:
def calc_saccades_amplitude(fixation_positions):
    x = np.array(fixation_positions)[:,0]
    y = np.array(fixation_positions)[:,1]
    dx = np.subtract(x[1:], x[:-1])
    dy = np.subtract(y[1:], y[:-1])
    distance = np.sum(np.sqrt(np.square(dx) + np.square(dy)))
    return distance

def fixations_info_uneye(data):
    fixations_duration_means = list()
    saccade_amplitudes = list()
    fixations_numbers = list()
    
    for i in range(len(data)):
        sample = data.iloc[i,2:].dropna()
        x = np.array(sample[0::2], dtype=float)
        y = np.array(sample[1::2], dtype=float)
        model = uneye.DNN(weights_name="weights_dataset3", sampfreq= SAMPLEING_FREQUENCY)
        uneye_predictions = model.predict(x, y)[0]
        uneye_fixations_position,  uneye_fixations_duration = get_fixations(x, y, uneye_predictions, SAMPLEING_FREQUENCY)

        fixations_duration_means.append(uneye_fixations_duration.mean())
        saccade_amplitudes.append(calc_saccades_amplitude(uneye_fixations_position))
        fixations_numbers.append(len(uneye_fixations_duration))
    return np.array(fixations_duration_means), np.array(saccade_amplitudes), np.array(fixations_numbers)

def fixations_info_vel_algo(data):
    fixations_duration_means = list()
    saccade_amplitudes = list()
    fixations_numbers = list()

    for i in range(len(data)):
        sample = data.iloc[i,2:].dropna()
        x = np.array(sample[0::2], dtype=float)
        y = np.array(sample[1::2], dtype=float)

        vel_algo_fixations_postion , vel_algo_fixations_duration = fixation_velocity(x,y, SAMPLEING_FREQUENCY)

        fixations_duration_means.append(vel_algo_fixations_duration.mean())
        saccade_amplitudes.append(calc_saccades_amplitude(vel_algo_fixations_postion))
        fixations_numbers.append(len(vel_algo_fixations_duration))
    return np.array(fixations_duration_means), np.array(saccade_amplitudes), np.array(fixations_numbers)


uneye_fixations_duration_means, uneye_saccade_amplitudes, uneye_fixations_numbers = fixations_info_uneye(data)
vel_alg_fixations_duration_means, vel_alg_saccade_amplitudes, vel_alg_fixations_numbers = fixations_info_vel_algo(data)
print("UnEye fixations duration mean: {}  Velocity Algorithm fixations duration mean: {}".format(uneye_fixations_duration_means.mean(), vel_alg_fixations_duration_means.mean()))
print("UnEye fixations number mean: {}  Velocity Algorithm fixations number mean: {}".format(uneye_fixations_numbers.mean(), vel_alg_fixations_numbers.mean()))

        


UnEye fixations duration mean: 0.13187554309401145  Velocity Algorithm fixations duration mean: 0.14490760825112622
UnEye fixations number mean: 15.32972972972973  Velocity Algorithm fixations number mean: 15.043243243243243


In [43]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='UnEye',
    x= ['Fixations duration (seconds)'], 
    y= [uneye_fixations_duration_means.mean()],
    error_y=dict(type='data', array=[uneye_fixations_duration_means.std()])
))
fig.add_trace(go.Bar(
    name='Velocity Algorithm',
    x= ['Fixations duration (seconds)'], 
    y= [vel_alg_fixations_duration_means.mean()],
    error_y=dict(type='data', array=[vel_alg_fixations_duration_means.std()])
))
fig.update_layout(barmode='group', autosize=False, legend=dict(x=0.3,y=-0.3))
fig.show()

#############################################

fig = go.Figure()
fig.add_trace(go.Bar(
    name='UnEye',
    x= ['Fixations number'], 
    y= [uneye_fixations_numbers.mean()],
    error_y=dict(type='data', array=[uneye_fixations_numbers.std()])
))
fig.add_trace(go.Bar(
    name='Velocity Algorithm',
    x= ['Fixations number'], 
    y= [vel_alg_fixations_numbers.mean()],
    error_y=dict(type='data', array=[vel_alg_fixations_numbers.std()])
))
fig.update_layout(barmode='group', autosize=False, legend=dict(x=0.3,y=-0.3))
fig.show()

#############################################

fig = go.Figure()
fig.add_trace(go.Bar(
    name='UnEye',
    x= ['Saccades amplitude (in pixles)'], 
    y= [uneye_saccade_amplitudes.mean()],
    error_y=dict(type='data', array=[uneye_saccade_amplitudes.std()])
))
fig.add_trace(go.Bar(
    name='Velocity Algorithm',
    x= ['Saccades amplitude (in pixles)'], 
    y= [vel_alg_saccade_amplitudes.mean()],
    error_y=dict(type='data', array=[vel_alg_saccade_amplitudes.std()])
))
fig.update_layout(barmode='group', autosize=False, legend=dict(x=0.3,y=-0.3))
fig.show()


# Comparing individual subjects (known=true and known=false)

### Getting data

In [44]:
MFD_true = []
MFD_SD_true = []
MFD_false = []
MFD_SD_false = []
MSA_true = []
MSA_SD_true = []
MSA_false = []
MSA_SD_false = []
MFD_overall = []
MFD_overall_SD = []
MSA_overall = []
MSA_overall_SD = []
for subject in subjects:
    dur_mean_true, amp_true, _ = fixations_info_vel_algo(data[(data.iloc[:, 0] == subject) & (data.iloc[:, 1] == True)])
    MFD_true.append(dur_mean_true.mean())
    MFD_SD_true.append(dur_mean_true.std())
    MSA_true.append(amp_true.mean())
    MSA_SD_true.append(amp_true.std())

    dur_mean_false, amp_false, _ = fixations_info_vel_algo(data[(data.iloc[:, 0] == subject) & (data.iloc[:, 1] == False)])
    MFD_false.append(dur_mean_false.mean())
    MFD_SD_false.append(dur_mean_false.std())
    MSA_false.append(amp_false.mean())
    MSA_SD_false.append(amp_false.std())

    dur_mean_overall, amp_overall, _ = fixations_info_vel_algo(data[data.iloc[:, 0] == subject])
    MFD_overall.append(dur_mean_overall.mean())
    MFD_overall_SD.append(dur_mean_overall.std())
    MSA_overall.append(amp_overall.mean())
    MSA_overall_SD.append(amp_overall.std())

subjects_info = np.stack([subjects, MFD_true, MFD_SD_true, MFD_false, MFD_SD_false, MSA_true, MSA_SD_true, MSA_false, MSA_SD_false, MFD_overall, MFD_overall_SD, MSA_overall, MSA_overall_SD]).T
subjects_info = pd.DataFrame(subjects_info, columns=["subject", "MFD_true", "MFD_SD_true", "MFD_false", "MFD_SD_false", "MSA_true", "MSA_SD_true", "MSA_false", "MSA_SD_false", "MFD_overall", "MFD_overall_SD", "MSA_overall", "MSA_overall_SD"])

subjects_info.to_csv("data.csv")
print(subjects_info)



  subject             MFD_true           MFD_SD_true            MFD_false  \
0      s2  0.17215175213675218   0.05290214167900286  0.14350833068977806   
1      s4   0.1567630527951581   0.04905050528454397  0.16685579511057197   
2      s8  0.11756709355459355   0.02383227158169356  0.10143515207015208   
3     s10  0.10396192982456141  0.012145660711345123  0.12851840205482137   
4     s16   0.1845480320445838  0.011302850197349893  0.19859680830039528   
5     s22   0.1314945165945166   0.03220975429930178  0.14186258438531166   
6     s23  0.14859523809523809  0.015002985341624444  0.20506376754558572   
7     s30  0.12139845291962939   0.04267092843711061  0.11483590417173731   

           MFD_SD_false            MSA_true         MSA_SD_true  \
0  0.061172214716787146   5435.837155781379  3221.2006963872577   
1   0.05217892900235616   7027.879347254966     5607.1738407395   
2  0.015457490412638525  2889.6622056777815   749.6302463742393   
3   0.03008888962445896   4977.3612935

### Duration of fixation

In [45]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Fixation duration (in seconds) for recognized images',
    x= subjects_info['subject'], y=subjects_info['MFD_true'].astype(float),
    error_y=dict(type='data', array=subjects_info['MFD_SD_true'].astype(float))
))
fig.add_trace(go.Bar(
    name='Fixation duration (in seconds) for non-recognized images',
    x= subjects_info['subject'], y=subjects_info['MFD_false'].astype(float),
    error_y=dict(type='data', array=subjects_info['MFD_SD_false'].astype(float))
))
fig.update_layout(barmode='group', legend=dict(x=0.3,y=-0.3))
fig.show()

In [46]:

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Fixation duration (in seconds) for recognized images',
    x= ["All subjects"], y=[subjects_info['MFD_true'].astype(float).to_numpy().mean()],
    error_y=dict(type='data', array=[subjects_info['MFD_SD_true'].astype(float).to_numpy().mean()])
))
fig.add_trace(go.Bar(
    name='Fixation duration (in seconds) for non-recognized images',
    x= ["All subjects"], y=[subjects_info['MFD_false'].astype(float).to_numpy().mean()],
    error_y=dict(type='data', array=[subjects_info['MFD_SD_false'].astype(float).to_numpy().mean()])
))

fig.update_layout(barmode='group', autosize=False, legend=dict(x=0.1,y=-0.3))
fig.show()


### Amplitude of Saccades

In [47]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Saccades Amplitudes (in pixels) for recognized images',
    x= subjects_info['subject'], y=subjects_info['MSA_true'].astype(float),
    error_y=dict(type='data', array=subjects_info['MSA_SD_true'].astype(float))
))
fig.add_trace(go.Bar(
    name='Saccades Amplitudes (in pixels) for non-recognized images',
    x= subjects_info['subject'], y=subjects_info['MSA_false'].astype(float),
    error_y=dict(type='data', array=subjects_info['MSA_SD_false'].astype(float))
))

fig.update_layout(legend=dict(x=0.3,y=-0.3))
fig.show()

In [48]:

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Saccade Amplitude (in pixels) for recognized images',
    x= ["All subjects"], y=[subjects_info['MSA_true'].astype(float).to_numpy().mean()],
    error_y=dict(type='data', array=[subjects_info['MSA_SD_true'].astype(float).to_numpy().mean()])
))
fig.add_trace(go.Bar(
    name='Saccade Amplitude (in pixels) for non-recognized images',
    x= ["All subjects"], y=[subjects_info['MSA_false'].astype(float).to_numpy().mean()],
    error_y=dict(type='data', array=[subjects_info['MSA_SD_false'].astype(float).to_numpy().mean()])
))

fig.update_layout(barmode='group', autosize=False, legend=dict(x=0.1,y=-0.3))
fig.show()
